In [10]:
import os
import random
import imageio
import glob
import datetime
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Rescaling
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tqdm.keras import TqdmCallback
import tensorflow_addons as tfa

from keras import layers
from keras import models
from keras.models import Model


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score


In [31]:
dataset_name = 'public_dataset'
batch_size = 8

In [32]:
image_size = (255, 255)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_name,
    validation_split=0.15,
    subset="training",
    label_mode = 'categorical',
    labels = 'inferred',
    seed=1336,
    image_size=image_size,
    interpolation = 'bilinear',
    batch_size=batch_size,
    shuffle = True,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_name,
    validation_split=0.15,
    subset="validation",
    label_mode = 'categorical',
    labels = 'inferred',
    seed=1336,
    image_size=image_size,
    interpolation = 'bilinear',
    batch_size=batch_size,
    shuffle = True,
)


Found 14910 files belonging to 3 classes.
Using 12674 files for training.
Found 14910 files belonging to 3 classes.
Using 2236 files for validation.


In [33]:
num_classes = 3

In [34]:
base_model = EfficientNetB0(include_top = False ,weights='imagenet', pooling='avg')

# Introduce a layer of data augmentation
# data_augmentation = Sequential([
#     preprocessing.RandomRotation(0.2),
    # preprocessing.RandomFlip("horizontal"),
    # preprocessing.RandomZoom(0.2),
    # preprocessing.RandomContrast(0.2),
    # preprocessing.RandomTranslation(0.2, 0.2),
    # preprocessing.RandomHeight(0.2),
    # preprocessing.RandomWidth(0.2),
# ])

# # Freeze all layers in the base model
# for layer in base_model.layers:
#     layer.trainable = False
# # Unfreeze the last 10 layers in the base model for fine-tuning
# for layer in base_model.layers[-5:]:
#     layer.trainable = True

#capa de entradas. 
entradas = layers.Input((255, 255, 3))

# Capa de augmentation
# x = data_augmentation(entradas)
# Pass the augmented images through the base model
x = base_model(entradas)
# Add a dense layer
x = layers.Dense(128, activation='relu')(x)
# Add another dense layer
salidas = layers.Dense(num_classes, activation='softmax')(x)
model1 = Model(inputs = entradas, outputs = salidas)
model1.summary()



Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 255, 255, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense_8 (Dense)              (None, 128)               163968    
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 387       
Total params: 4,213,926
Trainable params: 4,171,903
Non-trainable params: 42,023
_________________________________________________________________


In [35]:
import datetime

In [36]:
MODEL_NAME = 'Ef0'
RUN_NAME = 'public_dataset'
log_dir = f'new_logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}/{MODEL_NAME}{RUN_NAME}'


In [37]:
num_epochs = 10
patience = 5
patience_lr = 5

model1.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=[
                    tf.keras.metrics.CategoricalAccuracy(name=f'metrics/accuracy'),
                    tf.keras.metrics.TopKCategoricalAccuracy(3, name=f'metrics/top-3-accuracy'),
                    tfa.metrics.F1Score(num_classes=num_classes, average='macro', name='metrics/F1-macro'),
                    tf.keras.metrics.AUC(multi_label=True, num_labels=num_classes, name='metrics/AUC'),
                    tf.keras.metrics.Precision(name='metrics/precision'),
                    tf.keras.metrics.Recall(name='metrics/recall'),
                    tf.keras.metrics.PrecisionAtRecall(0.99, name='metrics/P@R_99'),
                    tf.keras.metrics.PrecisionAtRecall(0.95, name='metrics/P@R_95'),
                    tf.keras.metrics.PrecisionAtRecall(0.9, name='metrics/P@R_90'),
                    tfa.metrics.MatthewsCorrelationCoefficient(num_classes=num_classes, name='metrics/MCC')
                ],
             )

callbacks =[
       EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=patience),
       ReduceLROnPlateau(monitor='val_loss', patience=patience_lr, min_lr=1e-7),       
       ModelCheckpoint(log_dir, monitor=f"val_loss", save_best_only=True, save_weights_only=True),
       TqdmCallback(),
       TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

]

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [38]:
history = model1.fit(train_ds, epochs=num_epochs, verbose=1, callbacks=callbacks, validation_data=val_ds)

Epoch 1/80
1585/1585 [==============================] - 720s 431ms/step - loss: 0.1907 - metrics/accuracy: 0.9318 - metrics/top-3-accuracy: 1.0000 - metrics/F1-macro: 0.9316 - metrics/AUC: 0.9869 - metrics/precision: 0.9350 - metrics/recall: 0.9289 - metrics/P@R_99: 0.7460 - metrics/P@R_95: 0.9101 - metrics/P@R_90: 0.9539 - metrics/MCC: 0.8976 - val_loss: 0.0815 - val_metrics/accuracy: 0.9705 - val_metrics/top-3-accuracy: 1.0000 - val_metrics/F1-macro: 0.9705 - val_metrics/AUC: 0.9980 - val_metrics/precision: 0.9705 - val_metrics/recall: 0.9705 - val_metrics/P@R_99: 0.9233 - val_metrics/P@R_95: 0.9806 - val_metrics/P@R_90: 0.9907 - val_metrics/MCC: 0.9560
Epoch 2/80
1585/1585 [==============================] - 466s 294ms/step - loss: 0.0941 - metrics/accuracy: 0.9675 - metrics/top-3-accuracy: 1.0000 - metrics/F1-macro: 0.9674 - metrics/AUC: 0.9959 - metrics/precision: 0.9676 - metrics/recall: 0.9674 - metrics/P@R_99: 0.9026 - metrics/P@R_95: 0.9789 - metrics/P@R_90: 0.9884 - metrics/MC

KeyboardInterrupt: 